In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Estimator
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import PySCFDriver
from qiskit_nature.transformers.second_quantization import FreezeCoreTransformer
from qiskit_nature.circuit.library import UCCSD
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms import VQE
from qiskit.utils import QuantumInstance
from qiskit import Aer

# Initialize Qiskit Runtime Service
service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='ibm-q/open/main',
    token='your-token'
)

# Define the molecule
molecule = Molecule(
    geometry=[
        ["H", [0.0, 0.0, 0.0]],
        ["H", [0.0, 0.0, 0.74]]
    ],
    charge=0,
    multiplicity=1
)

# Create the driver
driver = PySCFDriver.from_molecule(molecule)
driver_result = driver.run()

# Apply freeze core transformer
transformer = FreezeCoreTransformer()
transformed_result = transformer.transform(driver_result)

# Define the ansatz
ansatz = UCCSD(
    qubit_converter=transformed_result.qubit_converter,
    num_particles=transformed_result.num_particles,
    num_spatial_orbitals=transformed_result.num_spatial_orbitals,
    initial_state=TwoLocal(rotation_blocks=['ry'], entanglement_blocks='cz')
)

# Set up VQE
quantum_instance = QuantumInstance(Aer.get_backend('statevector_simulator'))
vqe = VQE(ansatz, quantum_instance=quantum_instance)

# Run the VQE algorithm
with Session(service=service):
    estimator = Estimator()
    job = vqe.compute_minimum_eigenvalue(estimator=estimator)
    result = job.result()

# Print the results
print("Ground state energy:", result.eigenvalue)
